In [1]:
#importing relevant files 
import pickle
import re
import numpy as np
with open ('Pickle_Files/user_filepaths', 'rb') as fp:
    filepaths = pickle.load(fp)
print(len(filepaths))
users=[re.sub(r'\.txt$','',filename) for filename in filepaths]
print(users[0:9])

with open("Pickle_Files/preprocessed_user2.txt", "rb") as fp:   # Unpickling
    total_tokens_loaded= pickle.load(fp)
vectors_array=np.loadtxt('user_glove_vecs').view(float)
print(len(vectors_array))
print(len(total_tokens_loaded))

705
['100306237', '1005383936', '1009689475', '1017189240', '104048643', '1049020446', '1050455592', '105498404', '1057244852']
173781
705


In [2]:
#creating/importing dictionary and corpus
import gensim 
from gensim import corpora,models,similarities

"""
To create from scratch: 
dictionary=corpora.Dictionary(total_tokens_loaded)
dictionary.save_as_text('gensim_dict')
corpora.MmCorpus.serialize('lda_test2.mm', corpus)"""

dictionary=corpora.Dictionary.load_from_text('gensim_dict')
corpus=[dictionary.doc2bow(text) for text in total_tokens_loaded]
mm = corpora.MmCorpus('lda_test2.mm')


In [3]:
#running and saving model 
"""lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary, num_topics=10, update_every=1, chunksize=10, passes=1)
lda.save('LDA_model')"""

"lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary, num_topics=10, update_every=1, chunksize=10, passes=1)\nlda.save('LDA_model')"

In [4]:
#reloading model
lda =  models.LdaModel.load('LDA_model')
lda.print_topics(10)

[(0,
  '0.024*"co" + 0.013*"follow" + 0.011*"connecting" + 0.008*"Canada" + 0.007*"fucking" + 0.005*"school" + 0.005*"remember" + 0.005*"statement" + 0.005*"mission" + 0.005*"wt"'),
 (1,
  '0.012*"free" + 0.009*"ever" + 0.008*"co" + 0.007*"twitter" + 0.006*"could" + 0.006*"cot" + 0.005*"photo" + 0.005*"going" + 0.005*"ii" + 0.005*"job"'),
 (2,
  '0.193*"co" + 0.021*"via" + 0.012*"catholic" + 0.011*"profile" + 0.007*"la" + 0.007*"smiler" + 0.006*"syromalabar" + 0.006*"faith" + 0.006*"f" + 0.006*"digitalkalyan"'),
 (3,
  '0.009*"ward" + 0.009*"climate change" + 0.006*"goggle" + 0.006*"red" + 0.005*"gustavorejivik" + 0.005*"_tmf" + 0.005*"brandonrgates" + 0.005*"friends\'" + 0.005*"drwaheeduddin" + 0.005*"eddarrell"'),
 (4,
  '0.013*"you tube" + 0.006*"followed" + 0.005*"co" + 0.005*"revolution" + 0.005*"syromalabar" + 0.005*"thechucknellis" + 0.004*"checked" + 0.004*"digitalkalyan" + 0.004*"pastor" + 0.004*"indiancatholic"'),
 (5,
  '0.052*"Jesus" + 0.042*"god" + 0.020*"bible" + 0.014*"s

In [5]:
prob_topic=lda.get_document_topics(mm,minimum_probability=.1)

In [6]:
prob_topic[0]
# not too convenient for matrix manipulations

[(1, 0.34540814997381503), (4, 0.17996004861790998), (6, 0.13467681019016303)]

In [7]:
#creating normtheta
import numpy as np
from numpy import linalg as LA

gamma,_=lda.inference(mm)

### Need to normalize gamma

norm=gamma.sum(axis=1)
l=np.shape(norm)[0]
normtheta=gamma/norm.reshape(l,1)

normtheta[0,:]

# See it matches prob_topic

array([ 0.05408934,  0.34540787,  0.09592673,  0.01283011,  0.17995891,
        0.08252177,  0.13467666,  0.02459676,  0.04408647,  0.02590539])

In [8]:
#jensen-shannon
import scipy.stats as scst

def jsd(P,Q):
    M=(P+Q)/2
    return (scst.entropy(P,M)+scst.entropy(Q,M))/2
def jsd_mat(prob_mat):
    ndoc=np.shape(prob_mat)[0]
    jsdist=np.zeros((ndoc,ndoc))
    for i in range(ndoc):
        for j in range(ndoc):
            jsdist[i,j]=jsd(prob_mat[i,:],prob_mat[j,:])
    return jsdist

In [9]:
#creating&saving user dist 
"""dist_user=np.sqrt(2*jsd_mat(normtheta))
import numpy as np
np.savetxt('dist_user',dist_user.view(float))"""

"dist_user=np.sqrt(2*jsd_mat(normtheta))\nimport numpy as np\nnp.savetxt('dist_user',dist_user.view(float))"

In [10]:
#loading dist_user
import numpy as np
dist_user=np.loadtxt('dist_user').view(float)

In [11]:
print(len(total_tokens_loaded))

705


In [12]:
import pickle 
with open('Pickle_Files/user_dict_train.pickle', 'rb') as handle:
    user_dict_train = pickle.load(handle)
with open('Pickle_Files/user_dict_test.pickle', 'rb') as handle1:
    user_dict_test = pickle.load(handle1)
with open('Pickle_Files/user_dict_total.pickle', 'rb') as handle2:
    user_dict_total = pickle.load(handle2)

In [13]:
#user statistics
targets=['Legalization of Abortion', 'Climate Change is a Real Concern','Hillary Clinton','Feminist Movement','Atheism']
stances=['AGAINST','NONE','FAVOR']
count_topic_users={}
for target in targets:
    count_topic_users[target]={}
    for stance in stances:
        count_topic_users[target][stance]=0
#print(count_topic_users)

for user in user_dict_total: 
    if user in users:
        for i in range(len(targets)):
            #if len(user_dict_total[user][targets[i]])>0:
                #count_topic_users[targets[i]]+=1
                for j in range(len(stances)):
                    if stances[j] in user_dict_total[user][targets[i]]:
                        count_topic_users[targets[i]][stances[j]]+=1
                   
count_topic_users

{'Atheism': {'AGAINST': 120, 'FAVOR': 22, 'NONE': 22},
 'Climate Change is a Real Concern': {'AGAINST': 1, 'FAVOR': 66, 'NONE': 22},
 'Feminist Movement': {'AGAINST': 90, 'FAVOR': 41, 'NONE': 36},
 'Hillary Clinton': {'AGAINST': 129, 'FAVOR': 22, 'NONE': 24},
 'Legalization of Abortion': {'AGAINST': 115, 'FAVOR': 16, 'NONE': 41}}